# Implementation of the Gibbs sampler from the section 3

In [22]:
from particles import state_space_models as ssm
from particles import distributions as dists
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
from scipy import linalg
import particles
from particles import distributions
from scipy import stats
from particles import mcmc

On creer notre state space modele lorsque conditionnellement à z et zeta. Deplus ce state space modèle est défini unique pour un actif car conditionnelement à z et zeta nos actif sont independant.

In [23]:
class IdentifiedLoadingSSM(ssm.StateSpaceModel):
    """
    Modèle espace d'état pour les chargements d'un actif 'i' avec contraintes d'identification.
    
    Règles d'identification :
    - Pour les p premières lignes (i < p):
        * La matrice est triangulaire inférieure (lambda_{ik} = 0 pour k > i).
        * Les éléments diagonaux (lambda_{ii}) sont stricts positifs -> modélisés en log.
    - Pour les lignes suivantes (i >= p):
        * Pas de contrainte de structure ou de signe.
        
    Attributs:
        row_idx (int): Indice de l'actif courant (i).
        p_factors (int): Nombre total de facteurs.
        dim_state (int): Dimension effective du vecteur d'état pour cet actif.
    """
    def __init__(self, row_idx, p_factors, mu, phi, sigma_eta, z, zeta, data_x):
        self.i = row_idx
        self.p = p_factors
        
        # Dimension effective: Pour i < p, on ne simule que les i+1 premiers éléments.
        self.dim_state = min(row_idx + 1, p_factors)
        
        # On ne conserve que les paramètres pertinents pour la dimension active
        self.mu = mu[:self.dim_state]         # (dim_state,)
        self.phi = phi[:self.dim_state]       # (dim_state,)
        self.sigma_eta = sigma_eta[:self.dim_state] # (dim_state,)
        
        self.z = z       # Facteurs communs (T, p)
        self.zeta = zeta # Variable de mélange (T,)
        self.data_x = data_x # Données observées (T,)

    def PX0(self):
        """Distribution initiale des états h_0 """
        # Variance stationnaire: sigma^2 / (1 - phi^2)
        var_0 = self.sigma_eta**2 / (1 - self.phi**2)
        
        # Distribution indépendante sur chaque composante du vecteur d'état
        return distributions.IndepProd(
            distributions.Normal(loc=self.mu, scale=np.sqrt(var_0))
        )

    def PX(self, t, xp):
        """Transition des états h_t | h_{t-1} """
        # Dynamique AR(1) Gaussienne sur h_t
        # Note: Si k=i (diagonale), h_t correspond au LOG-loading.
        mean = self.mu + self.phi * (xp - self.mu)
        return distributions.IndepProd(
            distributions.Normal(loc=mean, scale=self.sigma_eta)
        )

    def PY(self, t, xp, x):
        """
        Vraisemblance p(x_it | h_it, z_t, zeta_t) 
        Transforme l'état latent h_t en chargement lambda_t puis calcule la densité.
        """
        # x est de forme (N_particules, dim_state)
        N = x.shape[0]
        
        # 1. Reconstitution du vecteur de chargements complet (taille p)
        real_loadings = np.zeros((N, self.p)) 
        
        # Copie des états latents
        real_loadings[:, :self.dim_state] = x
        
        # 2. Application de la contrainte de positivité (Log -> Exp)
        # Uniquement pour les éléments diagonaux des p premiers actifs
        if self.i < self.p:
            # L'élément diagonal est le dernier de l'état actif (indice self.i)
            # h_{ii} = log(lambda_{ii})  =>  lambda_{ii} = exp(h_{ii})
            real_loadings[:, self.i] = np.exp(x[:, self.i])
            
            # Les éléments k > i restent à 0.0 (Triangulaire inf)
            
        # 3. Calcul de la composante factorielle et du scaling
        z_t = self.z[t] # (p,)
        
        # Produit scalaire lambda' * z
        factor_comp = np.dot(real_loadings, z_t) # (N,)
        
        # Norme au carré des lambda pour le scaling sigma_{it} 
        lam_sq = np.sum(real_loadings**2, axis=1) # (N,)
        
        # Scaling factor: sqrt(1 + lambda'lambda)
        scaling = np.sqrt(1.0 + lam_sq)
        
        # Écart-type résiduel: sigma = 1 / scaling
        sigma_eps = 1.0 / scaling
        
        # 4. Paramètres de la loi Normale de l'observation x_it
        # x_it = sqrt(zeta) * ( (lambda/scaling)'z + (1/scaling)*eps )
        zeta_sqrt = np.sqrt(self.zeta[t])
        
        mean_obs = zeta_sqrt * (factor_comp / scaling)
        scale_obs = zeta_sqrt * sigma_eps
        
        return distributions.Normal(loc=mean_obs, scale=scale_obs).logpdf(self.data_x[t])

On ne pourra pas directement utilisé le mcmc.ParticleGibbs de particules car l'un des principales aventage de l'augmented Gibbs est que conditionnelement au facteur on peut faire chaque actif est indépendant

In [ ]:
class FactorCopulaGibbs(mcmc.GenericGibbs):
    """
    Échantillonneur de Gibbs pour le modèle de Copule Factorielle Dynamique.
    Basé sur l'Appendix A de Creal & Tsay (2015).
    """
    
    def __init__(self, data, n_factors, prior, **kwargs):
        super().__init__(**kwargs)
        self.data = data # u_it
        self.n = data.shape[1]
        self.T = data.shape[0]
        self.p = n_factors
        
        # Initialisation des paramètres Theta 
        self.theta = prior.sample_initial() 
        # Contient: mu, phi, sigma, nu (degrees of freedom), beta
        
        # Initialisation des variables latentes 
        self.z = np.random.normal(size=(self.T, self.p)) # Facteurs communs
        self.zeta = np.ones((self.T, self.n))            # Variables de mélange

        # A. Les Chargements Physiques (Lambdas)
        # Ce sont les valeurs réelles utilisées dans l'équation d'observation (5).
        # Pour les éléments diagonaux, ce sont des valeurs strictement POSITIVES.
        self.lambdas = np.zeros((self.T, self.n, self.p)) 
        
        # B. Les États Latents Gaussiens (States)
        # Ce sont les variables h_t qui suivent la dynamique AR(1) de l'équation (2).
        # Pour la diagonale : latent_states = log(lambdas)
        # Pour le reste : latent_states = lambdas
        self.latent_states = np.zeros((self.T, self.n, self.p))
        
        # Initialisation cohérente (optionnelle mais recommandée pour éviter log(0))
        # On peut initialiser la diagonale à une petite valeur positive pour lambda
        for i in range(self.p):
            # Diagonale initiale lambda = 0.1 -> latent = log(0.1)
            self.lambdas[:, i, i] = 0.1
            self.latent_states[:, i, i] = np.log(0.1)

        # 5. Attributs pour le Particle Gibbs
        self.ssm_cls = IdentifiedLoadingSSM # On utilise la bonne classe identifiée
        self.lambda_path = None

    def step(self):
        """Une itération complète du Gibbs Sampler (Appendix A)"""
        
        # Step 1: Missing Data 
        # On travaille avec des données complètes pour cet exemple
        
        # Step 2: Variables de mélange (Zeta)
        self.update_zeta()
        
        # Step 3: Degrees of Freedom (nu)
        self.update_nu()
        
        # Step 4: Facteurs Communs (z_t)
        self.update_z()
        
        # Step 5: State Variables (Lambda)
        self.update_lambda_pg()
        
        # Step 6: Bruit VAR (Sigma) 
        self.update_sigma()
        
        # Step 7: Paramètres VAR (mu, Phi)
        self.update_mu_phi()
        
    def update_zeta(self):
        """
        Step 2: Independence Metropolis-Hastings pour zeta.
        
        """
        pass 

    def update_nu(self):
        """
        Step 3: Random-walk Metropolis pour nu.
        
        """
        pass

    def update_z(self):
        """
        Step 4: Gibbs standard pour z_t.
        Mise à jour des facteurs latents communs.
        
        Sources:
        - Distribution conditionnelle z_t ~ N(Mean, Var)
        - Calcul de sigma_it et lambda_tilde_it
        - Structure de l'inverse de la matrice de corrélation
        """
        T, n = self.data.shape
        p = self.p  # Nombre de facteurs
        
        # 1. Préparation des conteneurs
        new_z = np.zeros((T, p))
        I_p = np.eye(p)
        
        # On boucle sur le temps car les paramètres Lambda changent à chaque t
        for t in range(T):
            # --- A. Récupération des variables au temps t ---
            # Données x_it (imputées/transformées)
            x_t = self.data[t]  # (n,)
            
            # Chargements latents lambda_it 
            # self.lambdas est de forme (T, n, p)
            lambda_t = self.lambdas[t] # (n, p)
            
            # Variables de mélange zeta_t (pour Student-t/Grouped-t)
            # Si le modèle est Gaussien, zeta_t = 1 partout.
            zeta_t = self.zeta[t] # (n,)

            # --- B. Calcul des paramètres redimensionnés (Scaling) ---
            # Selon , les paramètres utilisés dans la copule 
            # (tilde_lambda et sigma) dépendent de l'état latent lambda.
            
            # Calcul de la norme au carré de lambda pour chaque série i
            # lambda_sq = sum(lambda_{it}^2)
            lam_sq_norm = np.sum(lambda_t**2, axis=1) # (n,)
            
            # Variance idiosyncratique sigma_{it}^2 
            # sigma^2 = 1 / (1 + lambda'lambda)
            sigma_sq_t = 1.0 / (1.0 + lam_sq_norm) # (n,)
            sigma_t = np.sqrt(sigma_sq_t)          # (n,)
            
            # Chargements redimensionnés tilde_lambda 
            # tilde_lambda = lambda / sqrt(1 + lambda'lambda) = lambda * sigma
            lambda_tilde_t = lambda_t * sigma_t[:, np.newaxis] # (n, p)
            
            # --- C. Construction de la "Donnée Transformée" ---
            # Selon Step 4 : x_dot = x / sqrt(zeta)
            # Cela normalise la variance induite par la variable de mélange
            x_dot_t = x_t / np.sqrt(zeta_t) # (n,)

            # --- D. Calcul de la Moyenne et Variance Postérieure ---
            # Le prior sur z_t est N(0, I_p).
            # La vraisemblance est x_dot ~ N(tilde_lambda * z, D)
            # où D est diagonale avec éléments sigma_sq_t.
            
            # Precision Matrix = I_p + C' D^-1 C
            # Ici C = lambda_tilde. D^-1 = diag(1/sigma^2).
            
            # Astuce numérique : tilde_lambda' D^-1 tilde_lambda
            # revient à : lambda' lambda (car tilde_lambda = lambda * sigma)
            # Preuve: (lambda*sigma)' * (1/sigma^2) * (lambda*sigma) = lambda' * lambda
            
            # Calcul de la matrice de précision du likelihood
            # precision_data = lambda_t.T @ lambda_t # Ce serait l'astuce
            # Mais restons fidèles à la notation de l'article pour la clarté :
            
            # D_inv est un vecteur (diagonale inverse)
            D_inv_diag = 1.0 / sigma_sq_t # (n,)
            
            # Terme C' D^-1
            # On multiplie chaque colonne de lambda_tilde par D_inv
            Ct_Dinv = lambda_tilde_t.T * D_inv_diag[np.newaxis, :] # (p, n)
            
            # Precision Postérieure = I + C' D^-1 C
            # terme entre crochets
            precision_post = I_p + Ct_Dinv @ lambda_tilde_t # (p, p)
            
            # Covariance Postérieure (Sigma_z)
            # On utilise Cholesky pour la stabilité numérique et pour le tirage ensuite
            # L = cholesky(Precision) -> Precision = L L.T
            try:
                L_prec = linalg.cholesky(precision_post, lower=True)
                # Pour inverser, on résout le système linéaire. 
                # Cov = Prec^-1.
                # Mais on a juste besoin de résoudre Mean = Cov * Terme_Lineaire
                # => Prec * Mean = Terme_Lineaire
            except linalg.LinAlgError:
                # Fallback en cas de problèmes numériques rares
                L_prec = linalg.cholesky(precision_post + 1e-6 * np.eye(p), lower=True)

            # Moyenne Postérieure (Mu_z)
            # Mean = Cov_post * (C' D^-1 x_dot)
            # terme de droite dans la moyenne
            linear_term = Ct_Dinv @ x_dot_t # (p,)
            
            # Résolution de : Precision * Mean = linear_term
            # On utilise cholesky_solve pour la rapidité
            mu_post = linalg.cho_solve((L_prec, True), linear_term)
            
            # --- E. Tirage aléatoire (Draw) ---
            # z_t = mu_post + chol(Cov_post) * epsilon
            # Note: chol(Cov) = inv(L_prec.T)
            
            epsilon = np.random.normal(size=p)
            
            # On résout L_prec.T * z_noise = epsilon pour obtenir z_noise ~ N(0, Cov)
            # C'est plus stable que d'inverser explicitement
            z_noise = linalg.solve_triangular(L_prec.T, epsilon, lower=False)
            
            new_z[t] = mu_post + z_noise

        # Mise à jour de l'attribut de la classe
        self.z = new_z

    def update_lambda_pg(self):
            """
            Step 5: Mise à jour des chargements factoriels via Particle Gibbs.
            
            Gère deux ensembles de variables :
            1. self.latent_states (h_t) : Variables Gaussiennes suivant l'AR(1).
            2. self.lambdas (lambda_t) : Chargements physiques avec contraintes (exp sur diagonale).
            
            Sources:
            - [cite: 302] Utilisation du Particle Gibbs pour échantillonner les trajectoires.
            - [cite: 309] Parallélisation possible sur 'i' (ici boucle séquentielle).
            - [cite: 343] Utilisation du Backward Sampling pour améliorer le mélange.
            -  Application des contraintes d'identification (Triangulaire + Diagonale Log-Normale).
            """
            T, n = self.data.shape
            p = self.p
            
            # A. Préparation des conteneurs (T, n, p)
            # Remplis de zéros par défaut (ce qui gère implicitement la partie triangulaire supérieure = 0)
            new_lambdas = np.zeros((T, n, p)) 
            new_states = np.zeros((T, n, p))
            
            # B. Récupération des paramètres (Broadcasting implicite géré par l'indexation [i])
            # On suppose que theta contient des arrays (n, p)
            mu_vec = self.theta['mu']      
            phi_vec = self.theta['phi']    
            sigma_vec = self.theta['sigma'] 
            
            # C. Boucle sur chaque actif i (Indépendance conditionnelle aux facteurs Z)
            for i in range(n):
                
                # 1. Données
                # Idéalement, utilisez ici self.transform_u_to_x() si implémenté, 
                # sinon on prend les données brutes stockées dans self.data
                data_i = self.data[:, i] 
                
                # 2. Instanciation du modèle SSM pour l'actif i
                # La classe IdentifiedLoadingSSM gère la dimension effective (dim_state)
                ssm_i = self.ssm_cls(
                    row_idx=i,
                    p_factors=p,
                    mu=mu_vec[i],        # Vecteur complet (n, p), le SSM prendra ce qu'il faut
                    phi=phi_vec[i],
                    sigma_eta=sigma_vec[i],
                    z=self.z,            # Facteurs communs conditionnels (T, p)
                    zeta=self.zeta[:, i], # Variable de mélange (T,)
                    data_x=data_i
                )
                
                # 3. Exécution du Particle Gibbs (SMC)
                # N=100 particules est suffisant grâce au Backward Sampling [cite: 345]
                fk_model = ssm.Bootstrap(ssm=ssm_i, data=data_i)
                pf = particles.SMC(fk=fk_model, N=100, store_history=True)
                pf.run()
                
                # 4. Backward Sampling (FFBSm)
                # Tire une trajectoire unique h_{1:T} depuis l'historique des particules
                # [cite: 334, 343] "Draw a path ... using backwards sampling"
                traj_list = pf.hist.backward_sampling(1)
                
                # Conversion liste -> array numpy (T, dim_state)
                # [:, :, 0] car backward_sampling renvoie une liste de arrays (M, dim) où M=1
                traj_arr = np.array(traj_list)[:, :, 0] 
                
                # 5. Stockage et Transformations
                dim = ssm_i.dim_state
                
                # --- Stockage des États Latents (Gaussiens) ---
                # Utilisés pour les étapes 6 & 7 (Update Mu, Phi, Sigma)
                new_states[:, i, :dim] = traj_arr
                
                # --- Stockage des Chargements Physiques ---
                # Utilisés pour l'étape 4 (Update Z) et le calcul de vraisemblance
                
                # Par défaut, on copie les valeurs
                new_lambdas[:, i, :dim] = traj_arr
                
                # Application de la contrainte d'identification pour les p premiers actifs
                if i < p:
                    # La diagonale (indice i) est modélisée en Log pour garantir la positivité
                    # h_{ii} = log(lambda_{ii}) => lambda_{ii} = exp(h_{ii})
                    # [cite: 8] "performed in logarithms to guarantee positive values"
                    new_lambdas[:, i, i] = np.exp(traj_arr[:, i])
                    
                    # Note: Les éléments new_lambdas[:, i, dim:] restent à 0.0 (Triangulaire inf)

            # D. Mise à jour finale des attributs de la classe
            self.lambdas = new_lambdas       
            self.latent_states = new_states

    def update_sigma(self):
        """
        Step 6: Tirage de la variance d'état Sigma (Inverse Gamma).
        
        Met à jour la variance des chocs de transition des états latents h_t.
        Utilise self.latent_states (Gaussian) et NON self.lambdas.
        
        Sources:
        - [cite: 25] Draw Sigma conditional on state variables Lambda.
        - [cite: 26] Full conditional posteriors are inverse gamma distributions.
        - [cite: 363] Prior utilisé: InvGamma(20, 0.25).
        """
        
        # 1. Récupération des États Latents (h_t)
        # C'est ici le changement critique : on utilise les états gaussiens
        states_next = self.latent_states[1:]  # h_{2:T} (T-1, n, p)
        states_curr = self.latent_states[:-1] # h_{1:T-1} (T-1, n, p)
        
        # 2. Préparation des paramètres (Broadcasting temporel)
        # mu et phi sont (n, p) -> deviennent (1, n, p)
        mu = self.theta['mu'][np.newaxis, :, :]
        phi = self.theta['phi'][np.newaxis, :, :]
        
        # 3. Calcul des résidus de la transition AR(1)
        # Equation (2): h_{t+1} = mu + Phi(h_t - mu) + eta_t
        states_pred = mu + phi * (states_curr - mu)
        
        # eta_t ~ N(0, Sigma)
        residuals = states_next - states_pred # (T-1, n, p)
        
        # 4. Calcul de la Somme des Carrés des Erreurs (SSE)
        # On condense la dimension temporelle (axis=0)
        sse = np.sum(residuals**2, axis=0) # (n, p)
        
        # 5. Paramètres du Postérieur Inverse Gamma
        # Prior [cite: 363]
        alpha_prior = 20.0
        beta_prior = 0.25
        
        T_eff = self.latent_states.shape[0] - 1
        
        # Règle de mise à jour conjuguée
        alpha_post = alpha_prior + (T_eff / 2.0)
        beta_post = beta_prior + (sse / 2.0)
        
        # 6. Échantillonnage
        # Astuce NumPy: X ~ Gamma(a, scale=1/b) => 1/X ~ InvGamma(a, b)
        gamma_draws = np.random.gamma(shape=alpha_post, scale=1.0/beta_post)
        
        # On évite la division par zéro (très improbable mais bonne pratique)
        new_sigma = 1.0 / np.maximum(gamma_draws, 1e-10)
        
        # Mise à jour
        self.theta['sigma'] = new_sigma

    def update_mu_phi(self):
        """
        Step 7: Tirage de mu et Phi conditionnellement aux états latents.
        
        Effectue une régression bayésienne sur les états latents h_t (self.latent_states).
        Gère la contrainte de stationnarité pour Phi via une Normale Tronquée.
        
        Sources:
        - [cite: 27] Draw mu, Phi conditional on state variables... acceptance sampling.
        -[cite: 164]...performed in logarithms (géré par l'usage de latent_states).
        - [cite: 362] Prior mu ~ N(0.4, 2).
        - [cite: 363] Prior Phi ~ N(0.985, 0.001) tronqué sur (-1, 1).
        """
        
        # 1. Récupération des États Latents (Gaussian h_t)
        # Changement critique: utilisation de latent_states
        h_curr = self.latent_states[:-1] # (T-1, n, p)
        h_next = self.latent_states[1:]  # (T-1, n, p)
        T_eff = h_curr.shape[0]
        
        # Variance actuelle sigma^2 (n, p)
        sigma_sq = self.theta['sigma'] 
        
        # -------------------------------------------------------
        # A. Mise à jour de MU (Moyenne Long Terme)
        # Modèle: (h_{t+1} - Phi*h_t) = (1 - Phi)*mu + eta_t
        # -------------------------------------------------------
        phi = self.theta['phi'] # (n, p)
        
        # Prior Mu [cite: 362]
        mu_prior_mean = 0.4
        mu_prior_var = 2.0
        mu_prior_prec = 1.0 / mu_prior_var
        
        # Pseudo-Variable dépendante Y
        y_mu = h_next - phi[np.newaxis, :, :] * h_curr # (T-1, n, p)
        
        # Regresseur constant X = (1 - Phi)
        X_mu = (1.0 - phi) # (n, p)
        
        # Précision et Moyenne pondérée (Likelihood terms)
        # Somme sur T des précisions: T * X^2 / sigma^2
        lik_prec_mu = T_eff * (X_mu**2) / sigma_sq
        
        # Somme sur T des (X * Y / sigma^2)
        lik_mean_term_mu = (X_mu / sigma_sq) * np.sum(y_mu, axis=0)
        
        # Postérieur Mu
        post_prec_mu = mu_prior_prec + lik_prec_mu
        post_var_mu = 1.0 / post_prec_mu
        post_mean_mu = post_var_mu * (mu_prior_prec * mu_prior_mean + lik_mean_term_mu)
        
        # Tirage Normal
        self.theta['mu'] = np.random.normal(loc=post_mean_mu, scale=np.sqrt(post_var_mu))
        
        # -------------------------------------------------------
        # B. Mise à jour de PHI (Autocorrélation)
        # Modèle: (h_{t+1} - mu) = Phi * (h_t - mu) + eta_t
        # -------------------------------------------------------
        mu = self.theta['mu'] # (n, p) (Nouvelle valeur)
        
        # Prior Phi [cite: 363]
        phi_prior_mean = 0.985
        phi_prior_var = 0.001
        phi_prior_prec = 1.0 / phi_prior_var
        
        # Centrage des données (X = h_t - mu)
        x_phi = h_curr - mu[np.newaxis, :, :]
        y_phi = h_next - mu[np.newaxis, :, :]
        
        # Statistiques suffisantes (Sommes des produits croisés)
        sum_x_sq = np.sum(x_phi**2, axis=0)      # sum(X^2)
        sum_xy = np.sum(x_phi * y_phi, axis=0)   # sum(X*Y)
        
        # Termes de vraisemblance
        lik_prec_phi = sum_x_sq / sigma_sq
        lik_mean_term_phi = sum_xy / sigma_sq
        
        # Postérieur Phi (Non Tronqué)
        post_prec_phi = phi_prior_prec + lik_prec_phi
        post_var_phi = 1.0 / post_prec_phi
        post_mean_phi = post_var_phi * (phi_prior_prec * phi_prior_mean + lik_mean_term_phi)
        post_std_phi = np.sqrt(post_var_phi)
        
        # Tirage Tronqué sur (-1, 1) pour stationnarité
        # Calcul des bornes normalisées pour truncnorm (a, b)
        # Z = (X - mean) / std
        a_clip = (-1.0 - post_mean_phi) / post_std_phi
        b_clip = (1.0 - post_mean_phi) / post_std_phi
        
        # Utilisation de scipy.stats.truncnorm (vectorisé)
        self.theta['phi'] = stats.truncnorm.rvs(
            a_clip, 
            b_clip, 
            loc=post_mean_phi, 
            scale=post_std_phi, 
            size=mu.shape
        )

In [26]:
class UnivariateLoadingSSM(ssm.StateSpaceModel):
    """
    Modèle espace d'état pour les chargements d'un seul actif i.
    Transition: Eq (2) [cite: 89]
    Observation: Eq (5) avec scaling Eq (6)-(7) [cite: 146, 156]
    """
    def __init__(self, mu, phi, sigma_eta, z, zeta, data_x):
        """
        :param mu: Moyenne long terme (scalaire ou vecteur taille p)
        :param phi: Autocorrélation (scalaire ou diag)
        :param sigma_eta: Variance du bruit de transition (scalaire ou diag)
        :param z: Facteurs communs (T, p) [cite: 150]
        :param zeta: Variable de mélange (T,) [cite: 220]
        :param data_x: Données observées transformées x_it (T,) [cite: 5]
        """
        self.mu = mu
        self.phi = phi
        self.sigma_eta = sigma_eta
        self.z = z
        self.zeta = zeta
        self.data_x = data_x
        
    def PX0(self):
        """Distribution initiale de lambda_{i,1} """
        # Stationnaire: N(mu, sigma^2 / (1-phi^2))
        var_0 = self.sigma_eta / (1 - self.phi**2)
        return distributions.Normal(loc=self.mu, scale=np.sqrt(var_0))

    def PX(self, t, xp):
        """Transition lambda_{i, t} | lambda_{i, t-1} (Eq 2) """
        # AR(1): mu + phi * (prev - mu) + noise
        mean = self.mu + self.phi * (xp - self.mu)
        return distributions.Normal(loc=mean, scale=np.sqrt(self.sigma_eta))

    def PY(self, t, xp, x):
        """
        Densité d'observation p(x_it | lambda_{it}, z_t, zeta_t)
        Utilise les équations de scaling (6) et (7).
        """
        # 1. Récupérer les covariables au temps t
        z_t = self.z[t]       # Facteurs (p,)
        zeta_t = self.zeta[t] # Variable de mélange (scalaire)
        obs = self.data_x[t]  # Donnée x_it
        
        # 2. Calcul du scaling 
        # Dans le cas p=1 (un seul facteur par loading pour simplifier l'exemple)
        # Si p > 1, x est un vecteur et il faut sommer x^2
        # Scaling factor: D = 1 + lambda'lambda
        # Note: x ici est la particule courante représentant lambda_{it}
        
        # Gestion vectorielle pour les particules (N_particles,)
        lam_sq = x**2 
        scaling_factor = np.sqrt(1.0 + lam_sq) 
        
        # 3. Chargements et variance transformés 
        # tilde_lambda = lambda / sqrt(1 + lambda^2)
        tilde_lambda = x / scaling_factor
        
        # sigma_eps = 1 / sqrt(1 + lambda^2)
        sigma_eps = 1.0 / scaling_factor
        
        # 4. Moyenne et Variance conditionnelles de x_it
        # x_it = sqrt(zeta) * (tilde_lambda * z_t + sigma_eps * epsilon)
        # Donc x_it ~ N( Mean, Var )
        
        # Moyenne = sqrt(zeta) * tilde_lambda * z_t
        mean_obs = np.sqrt(zeta_t) * tilde_lambda * z_t
        
        # Ecart-type = sqrt(zeta) * sigma_eps
        scale_obs = np.sqrt(zeta_t) * sigma_eps
        
        return distributions.Normal(loc=mean_obs, scale=scale_obs).logpdf(obs)